In [2]:
import cv2
import numpy as np  
from ultralytics import YOLO

import pygetwindow as gw
import win32gui
import win32con
import ctypes
import os

import time
import math 

import tkinter as tk

from threading import Thread
import threading 

## Detector de Operaciones activas

In [3]:
windows = gw.getAllTitles()

for i, title in enumerate(windows):
    if title.strip():
        print(f"{i}: {title}")

1: ● Interfaz-Instintiva.ipynb - Untitled (Workspace) - Visual Studio Code
2: #indeed | Drama Up - Discord
3: Binding of Isaac: Repentance+ v1.9.7.8
4: Steam
5: Proyecto
6: MarTonPerCar/Interfaz-Instintiva- - Opera
7: El Tiempo
8: El Tiempo
9: Reproductor multimedia
10: Reproductor multimedia
11: Configuración
12: Configuración
14: Calculadora
15: Calculadora
16: Microsoft Text Input Application
22: Anaconda Prompt
23: OBS 30.2.0 - Perfíl: Sin Título - Escenas: Sin Título
24: Program Manager


## Función Yolo Principal

### Consola

In [4]:
def crear_consola(altura, ancho):
    """
    Crea una imagen en blanco para usar como consola gráfica.

    Parámetros:
    - altura (int): Altura de la consola en píxeles.
    - ancho (int): Ancho de la consola en píxeles.
    """
    return np.ones((altura, ancho, 3), dtype=np.uint8) * 255

def escribir_en_consola(console_image, texto, y_offset=30):
    """
    Escribe líneas de texto en una consola gráfica.

    Parámetros:
    - console_image (np.ndarray): Imagen de la consola donde se escribe el texto.
    - texto (list of str): Lista de líneas de texto a escribir.
    - y_offset (int): Posición vertical inicial para escribir el texto.
    """
    for linea in texto:
        cv2.putText(console_image, linea, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        y_offset += 20

### Brazos

In [5]:
def calcular_angulo(x1, y1, x2, y2):
    """
    Calcula el ángulo en grados entre dos puntos.

    Parámetros:
    - x1, y1 (float): Coordenadas del primer punto.
    - x2, y2 (float): Coordenadas del segundo punto.
    """
    angulo_radianes = math.atan2((y2 - y1), (x2 - x1))
    angulo_grados = math.degrees(angulo_radianes)

    if angulo_grados < 0:
        angulo_grados += 360 
        
    return angulo_grados

def clasificar_angulo_dinamico(x1, y1, x2, y2, x3, y3, ca1, ca2, umbral_relativo=0.7):
    """
    Clasifica dinámicamente un ángulo en base a la distancia entre puntos y referencia.

    Parámetros:
    - x1, y1 (float): Coordenadas del punto móvil A (La mano).
    - x2, y2 (float): Coordenadas del punto B.
    - x3, y3 (float): Coordenadas del punto fijo C (El hombro).
    - ca1, ca2 (float): Coordenadas de la cadera.
    - umbral_relativo (float): Factor de tolerancia para clasificar como "Hacia Adelante".
    """
    distancia_referencia = math.sqrt((ca1 - x3)**2 + (ca2 - y3)**2)
    distancia_actual = math.sqrt((x1 - x3)**2 + (y1 - y3)**2)

    if distancia_actual <= distancia_referencia * umbral_relativo:
        return ["Recto", "Hacia Adelante", f"DA: {distancia_actual:.1f}, DR: {distancia_referencia:.1f}"]

    angulo_grados_1 = calcular_angulo(x2, y2, x3, y3)
    angulo_grados_2 = calcular_angulo(x1, y1, x2, y2)

    # Clasificar el ángulo
    if angulo_grados_1 >= 315 or angulo_grados_1 < 45:
        if  15 <= angulo_grados_2 < 90:
            return f"Derecha Arriba (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        elif 270 <= angulo_grados_2 < 345:
            return f"Derecha Abajo (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        else:
            return f"Derecha Recto (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"

    elif 45 <= angulo_grados_1 < 135:
        if 0 <= angulo_grados_2 < 75:
            return f"Arriba Derecha (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        elif 105 <= angulo_grados_2 < 180:
            return f"Arriba Izquierda (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        else:
            return f"Arriba Recto (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"

    elif 135 <= angulo_grados_1 < 225:
        if 90 <= angulo_grados_2 < 165:
            return f"Izquierda Arriba (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        elif 195 <= angulo_grados_2 < 270:
            return f"Izquierda Abajo (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        else:
            return f"Izquierda Recto (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"

    elif 225 <= angulo_grados_1 < 315:
        if 180 <= angulo_grados_2 < 255:
            return f"Abajo Izquierda (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        elif 285 <= angulo_grados_2 < 360:
            return f"Abajo Derecha (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"
        else:
            return f"Abajo Recto (Ángulo_1: {angulo_grados_1:.1f}, Ángulo_2: {angulo_grados_2:.1f})"

def calcular_distancia_brazo_izquierdo(keypoints, nombres_puntos):
    """
    Calcula la dirección del brazo izquierdo basado en ángulos y distancias.

    Parámetros:
    - keypoints (list): Lista de puntos clave detectados.
    - nombres_puntos (list): Nombres de los puntos clave.
    """
    try:
        hombro_izq = keypoints[nombres_puntos.index("Hombro izquierdo")]
        codo_izq = keypoints[nombres_puntos.index("Codo izquierdo")]
        muñeca_izq = keypoints[nombres_puntos.index("Muñeca izquierda")]
        cadera_izq = keypoints[nombres_puntos.index("Cadera izquierda")]

        if hombro_izq[2] <= 0 or codo_izq[2] <= 0 or muñeca_izq[2] <= 0:
            return "False"

        resultado = clasificar_angulo_dinamico(muñeca_izq[0], muñeca_izq[1], codo_izq[0], codo_izq[1], hombro_izq[0], hombro_izq[1], cadera_izq[0], cadera_izq[1])
        return resultado

    except (ValueError, IndexError) as e:
        return ["Recto", "Hacia Adelante", "Desconocido"]
    
def calcular_distancia_brazo_derecho(keypoints, nombres_puntos):
    """
    Calcula la dirección del brazo derecho basado en ángulos y distancias.

    Parámetros:
    - keypoints (list): Lista de puntos clave detectados.
    - nombres_puntos (list): Nombres de los puntos clave.
    """
    try:
        hombro_der = keypoints[nombres_puntos.index("Hombro derecho")]
        codo_der = keypoints[nombres_puntos.index("Codo derecho")]
        muñeca_der = keypoints[nombres_puntos.index("Muñeca derecha")]
        cadera_der = keypoints[nombres_puntos.index("Cadera derecha")]

        if hombro_der[2] <= 0 or codo_der[2] <= 0 or muñeca_der[2] <= 0:
            return "False"

        resultado = clasificar_angulo_dinamico(muñeca_der[0], muñeca_der[1], codo_der[0], codo_der[1], hombro_der[0], hombro_der[1], cadera_der[0], cadera_der[1])
        return resultado

    except (ValueError, IndexError):
        return ["Recto", "Hacia Adelante", "Desconocido"]

### Pierna

In [6]:
def calcular_angulo(x1, y1, x2, y2):
    """
    Calcula el ángulo entre dos puntos en grados.

    Parámetros:
    - x1, y1 (float): Coordenadas del primer punto.
    - x2, y2 (float): Coordenadas del segundo punto.
    """
    angulo_radianes = math.atan2((y2 - y1), (x2 - x1))
    angulo_grados = math.degrees(angulo_radianes)

    if angulo_grados < 0:
        angulo_grados += 360 
    return angulo_grados


def calcular_distancia_pierna_izquierda(keypoints, nombres_puntos):
    """
    Calcula la dirección y el ángulo de la pierna izquierda basada en puntos clave.

    Parámetros:
    - keypoints (list): Lista de puntos clave detectados.
    - nombres_puntos (list): Nombres de los puntos clave.
    """
    try:
        cadera_izq = keypoints[nombres_puntos.index("Cadera izquierda")]
        tobillo_izq = keypoints[nombres_puntos.index("Tobillo izquierdo")]

        if cadera_izq[2] <= 0.30 or tobillo_izq[2] <= 0.30:
            return ["Recto", "Ángulo: 270.00º"]

        angulo_ac = calcular_angulo(tobillo_izq[0], tobillo_izq[1], cadera_izq[0], cadera_izq[1])

        if 258<= angulo_ac <= 285:
            clasificacion = "Recto"
        elif angulo_ac < 258:
            clasificacion = "Izquierda"
        else:  # angulo_ac > 285
            clasificacion = "Derecha"

        return [clasificacion, f"Ángulo: {angulo_ac:.2f}°"]

    except (ValueError, IndexError):
        return ["Puntos clave insuficientes", ""]
    

def calcular_distancia_pierna_derecha(keypoints, nombres_puntos):
    """
    Calcula la dirección y el ángulo de la pierna derecha basada en puntos clave.

    Parámetros:
    - keypoints (list): Lista de puntos clave detectados.
    - nombres_puntos (list): Nombres de los puntos clave.
    """
    try:
        cadera_der = keypoints[nombres_puntos.index("Cadera derecha")]
        tobillo_der = keypoints[nombres_puntos.index("Tobillo derecho")]

        if cadera_der[2] <= 0.30 or tobillo_der[2] <= 0.30:
            return ["Recto", "Ángulo: 270.00º"]

        angulo_ac = calcular_angulo(tobillo_der[0], tobillo_der[1], cadera_der[0], cadera_der[1])

        if 255 <= angulo_ac <= 288:
            clasificacion = "Recto"
        elif angulo_ac < 255:
            clasificacion = "Izquierda"
        else:  # angulo_ac > 288
            clasificacion = "Derecha"

        return [clasificacion, f"Ángulo: {angulo_ac:.2f}°, {cadera_der[2]}, {tobillo_der[2]}"]

    except (ValueError, IndexError):
        return ["Recto", "Ángulo: 270.00º"]

### Check de la Nariz

In [7]:
def check_Nariz(keypoints, nombres_puntos):
    """
    Verifica la posición relativa de la nariz con respecto a los hombros para clasificarla como "Por debajo" o "Por arriba".

    Parámetros:
    - keypoints: Lista de coordenadas de los puntos clave detectados en el modelo.
    - nombres_puntos: Lista con los nombres de los puntos clave para indexarlos.
    """
    try:
        hombro_izq = keypoints[nombres_puntos.index("Hombro izquierdo")]
        hombro_der = keypoints[nombres_puntos.index("Hombro derecho")]
        nariz = keypoints[nombres_puntos.index("Nariz")]

        if nariz[2] <= 0.30 or hombro_izq[2] <= 0.30 or hombro_der[2] <= 0.30:
            return ["Por arriba"]

        if hombro_der[1] <= nariz[1] and hombro_izq[1] <= nariz[1]:
            return "Por debajo"
        else:
            return "Por arriba" 

    except (ValueError, IndexError):
        return ["Por arriba"]   

### Codigo Principal

In [8]:
def allow_set_foreground():
    """
    Permite que el proceso actual pueda establecer ventanas en primer plano.
    """
    ctypes.windll.user32.AllowSetForegroundWindow(-1)


def press_key(hexKeyCode, delay_time):
    """
    Simula la pulsación de una tecla específica utilizando su código hexadecimal.

    Parámetros:
    - hexKeyCode: Código hexadecimal de la tecla.
    - delay_time: Tiempo en segundos durante el cual se mantiene presionada la tecla.
    """
    ctypes.windll.user32.keybd_event(hexKeyCode, 0, 0, 0)
    time.sleep(delay_time)
    ctypes.windll.user32.keybd_event(hexKeyCode, 0, 0x0002, 0)


def focus_and_fullscreen(window_keyword):
    """
    Busca una ventana basada en un título parcial, centra el foco en ella y la restaura si está minimizada.

    Parámetros:
    - window_keyword: Parte del título de la ventana para buscar.
    """
    windows = [w for w in gw.getAllTitles() if window_keyword.lower() in w.lower()]
    if windows:
        window = gw.getWindowsWithTitle(windows[0])[0]
        win32gui.ShowWindow(window._hWnd, win32con.SW_RESTORE) 
        win32gui.SetForegroundWindow(window._hWnd)
        print(f"Ventana '{windows[0]}' encontrada")
        return True
    else:
        return False


game = "Binding of Isaac: Repentance+ v1.9.7.8"
#game = "JSB"

def run_menu():
    """
    Crea un menú gráfico con botones para iniciar y salir de la aplicación. 
    El botón "Iniciar" centra el foco en una ventana específica y lanza la cámara en un hilo separado.
    El botón "Debug" hace lo mismo que el Iniciar más algunos extras para ver que ocurre por detrá
    El botón "Salir" cierra la aplicación y envía un comando para cerrar la ventana de la cámara.
    """
    root = tk.Tk()
    root.title("Controlador del Juego")
    root.geometry("650x250") 
    root.configure(bg='light blue')

    title_label = tk.Label(root, text="Controlador Interactivo", font=("Helvetica", 16, "bold"), bg='light blue')
    title_label.pack(pady=20)

    def on_start_N():
        """
        Acción que ocurre al pulsar el botón "Iniciar". Centra la ventana del juego y ejecuta la cámara.
        """

        if focus_and_fullscreen('YOLOv8 - Camara'):
            time.sleep(0.25)
            press_key(0x51, 0.05)

        title_label["text"] = "La Camara y el juego se están encendiendo"
        start_button["state"] = "disabled"
        debug_button["state"] = "disabled"
        exit_button["state"] = "disabled"

        allow_set_foreground()

        camara_thread = threading.Thread(target=run_camara, daemon=True)
        camara_thread.start()
        schedule_check(camara_thread, "normal")

        game_thread_N = threading.Thread(target=run_game_N, daemon=True)
        game_thread_N.start()
            
    def on_start_D():
        """
        Acción que ocurre al pulsar el botón "Iniciar". Centra la ventana del juego y ejecuta la cámara.
        """

        if focus_and_fullscreen('YOLOv8 - Camara'):
            time.sleep(0.25)
            press_key(0x51, 0.05)

        title_label["text"] = "La Camara y el juego se están encendiendo en modo debug"
        start_button["state"] = "disabled"
        debug_button["state"] = "disabled"
        exit_button["state"] = "disabled"

        allow_set_foreground()

        camara_thread = threading.Thread(target=run_camara_debug, daemon=True)
        camara_thread.start()
        schedule_check(camara_thread, "debug")

        game_thread_D = threading.Thread(target=run_game_D, daemon=True)
        game_thread_D.start()


    def on_exit():
        """
        Acción que ocurre al pulsar el botón "Salir". Cierra la aplicación y envía comandos para cerrar la cámara.
        """
        root.destroy()
        if focus_and_fullscreen('YOLOv8 - Camara'):
            time.sleep(0.25)
            press_key(0x51, 0.05)

        if focus_and_fullscreen("Esqueleto Invertido"):
            time.sleep(0.25)
            press_key(0x51, 0.05)

        os._exit(0)

        
    def schedule_check(t, mode):
        """
        Programar la ejecución de la función `check_if_done()` dentro de un segundo.
        """
        if mode == "normal":
            root.after(1000, check_if_done_N, t)
        elif mode == "debug":
            root.after(1000, check_if_done_D, t)

    def check_if_done_N(t):
        if focus_and_fullscreen("YOLOv8 - Camara"):
            title_label["text"] = "La Camara y el juego están en funcionamiento"

            debug_button["state"] = "normal"
            exit_button["state"] = "normal"

        else:
            schedule_check(t, "normal")

    def check_if_done_D(t):
        if focus_and_fullscreen("Esqueleto Invertido"):
            title_label["text"] = "La Camara y el juego están en funcionamiento"

            print("Yeah")

            start_button["state"] = "normal"
            exit_button["state"] = "normal"
                
        else:
            schedule_check(t, "debug")

    start_button = tk.Button(root, text="Iniciar", command=on_start_N, bg='light gray', font=("Helvetica", 12))
    start_button.pack(pady=10)

    debug_button = tk.Button(root, text="Debugger", command=on_start_D, bg='light gray', font=("Helvetica", 12))
    debug_button.pack(pady=10)

    exit_button = tk.Button(root, text="Salir", command=on_exit, bg='red', font=("Helvetica", 12))
    exit_button.pack(pady=10)

    root.mainloop()

def run_camara():
    """
    Inicia la cámara ejecutando la función `Camara` en un hilo separado.
    """
    Camara("Normal")
    time.sleep(1)

def run_camara_debug():
    """
    Inicia la cámara ejecutando la función `Camara` en un hilo separado.
    """
    Camara("Debug")
    time.sleep(1)

def run_game_N():
    while not focus_and_fullscreen("YOLOv8 - Camara"):
        time.sleep(1)

    focus_and_fullscreen(game)    

    time.sleep(2)
    press_key(0x1B, 0.05)

def run_game_D():
    while not focus_and_fullscreen("Esqueleto Invertido"):
        time.sleep(1)

    focus_and_fullscreen(game)

    time.sleep(2)
    press_key(0x1B, 0.05)

In [9]:
active_keys_m = set()
last_direction_m = None

active_keys_t = set()
last_direction_t = None

def simulate_keys(value):
    """
    Simula la pulsación de una tecla específica según su valor.

    Parámetros:
    - value: Cadena que indica qué tecla simular ("ctrl", "esc", "q", "e", "space").
    """
    if value == "ctrl":
        press_key(0x11, 5)
    elif value == "esc":
        press_key(0x1B, 0.05)    
    elif value == "q":
        press_key(0x51, 0.05)
    elif value == "e":
        press_key(0x45, 0.05)
    elif value == "space":
        press_key(0x20, 0.05)

def press_and_hold_movement(keys_to_press, release_keys=None):
    """
    Mantiene presionadas las teclas especificadas para el movimiento, soltando las que ya no son necesarias.

    Parámetros:
    - keys_to_press: Lista de códigos de teclas que deben mantenerse presionadas.
    - release_keys: Lista de códigos de teclas a liberar (por defecto, las que ya no están activas).
    """
    global active_keys_m

    if release_keys is None:
        release_keys = active_keys_m - set(keys_to_press)

    for key in release_keys:
        ctypes.windll.user32.keybd_event(key, 0, 0x0002, 0)  # Soltar
        active_keys_m.discard(key)

    for key in keys_to_press:
        if key not in active_keys_m:
            ctypes.windll.user32.keybd_event(key, 0, 0, 0)  # Presionar
            active_keys_m.add(key)

def press_and_hold_tears(keys_to_press, release_keys=None):
    """
    Mantiene presionadas las teclas especificadas para los disparos (tears), soltando las que ya no son necesarias.

    Parámetros:
    - keys_to_press: Lista de códigos de teclas que deben mantenerse presionadas.
    - release_keys: Lista de códigos de teclas a liberar (por defecto, las que ya no están activas).
    """
    global active_keys_t

    if release_keys is None:
        release_keys = active_keys_t - set(keys_to_press)
    for key in release_keys:
        ctypes.windll.user32.keybd_event(key, 0, 0x0002, 0)  # Soltar
        active_keys_t.discard(key)

    for key in keys_to_press:
        if key not in active_keys_t:
            ctypes.windll.user32.keybd_event(key, 0, 0, 0)  # Presionar
            active_keys_t.add(key)

def pause(resultado_check_nariz, NewValue, Parado_value):
    """
    Controla la pulsación de la tecla ESC basada en la posición detectada de la nariz.

    Parámetros:
    - resultado_check_nariz: Cadena que indica si la nariz está "Por debajo" o "Por arriba".
    - NewValue: Lista con un único elemento que indica si ya se ha presionado ESC.
    - Parado_value: Lista con un único elemento que alterna entre 0 y 1 para indicar el estado de pausa.
    """
    if (resultado_check_nariz == "Por debajo") and (NewValue[0] == 0):
        simulate_thread = Thread(target=simulate_keys, args=('esc',))
        simulate_thread.start()
        
        NewValue[0] = 1

        if Parado_value[0] == 0:
            Parado_value[0] = 1

        elif Parado_value[0] == 1:
            Parado_value[0] = 0 

    elif (resultado_check_nariz == "Por arriba") and (NewValue[0] == 1):
        NewValue[0] = 0

    return NewValue, Parado_value

def pierna_izquierda_controls(resultado_cadera_pies_izquierda, contador_frame_izq, fps_2_seconds):
    """
    Controla las acciones del personaje basándose en los movimientos de la pierna izquierda.

    Parámetros:
    - resultado_cadera_pies_izquierda: Cadena que indica la dirección de la pierna ("Izquierda", "Derecha", "Recto").
    - contador_frame_izq: Diccionario que lleva el conteo de frames para cada dirección.
    - fps_2_seconds: Número de frames necesarios para confirmar una acción.
    """
    if (resultado_cadera_pies_izquierda == "Izquierda"):
        contador_frame_izq["Izquierda"] += 1
        contador_frame_izq["Derecha"] = 0

        if contador_frame_izq["Izquierda"] == fps_2_seconds:
            simulate_thread = Thread(target=simulate_keys, args=('ctrl',))
            simulate_thread.start()

    elif (resultado_cadera_pies_izquierda == "Derecha"):
        contador_frame_izq["Izquierda"] = 0
        contador_frame_izq["Derecha"] += 1

        if contador_frame_izq["Derecha"] == fps_2_seconds:
            simulate_thread = Thread(target=simulate_keys, args=('e',))
            simulate_thread.start()

    elif (resultado_cadera_pies_izquierda == "Recto"): 
        contador_frame_izq["Izquierda"] = 0
        contador_frame_izq["Derecha"] = 0      

    return contador_frame_izq

def pierna_derecha_controls(resultado_cadera_pies_derecha, contador_frame_der, fps_2_seconds):
    """
    Controla las acciones del personaje basándose en los movimientos de la pierna derecha.

    Parámetros:
    - resultado_cadera_pies_derecha: Cadena que indica la dirección de la pierna ("Izquierda", "Derecha", "Recto").
    - contador_frame_der: Diccionario que lleva el conteo de frames para cada dirección.
    - fps_2_seconds: Número de frames necesarios para confirmar una acción.
    """
    if (resultado_cadera_pies_derecha == "Izquierda"):
        contador_frame_der["Izquierda"] += 1
        contador_frame_der["Derecha"] = 0

        if contador_frame_der["Izquierda"] == fps_2_seconds:
            simulate_thread = Thread(target=simulate_keys, args=('q',))
            simulate_thread.start()

    elif (resultado_cadera_pies_derecha == "Derecha"):
        contador_frame_der["Izquierda"] = 0
        contador_frame_der["Derecha"] += 1

        if contador_frame_der["Derecha"] == fps_2_seconds:
            simulate_thread = Thread(target=simulate_keys, args=('space',))
            simulate_thread.start()

    elif (resultado_cadera_pies_derecha == "Recto"):
        contador_frame_der["Izquierda"] = 0
        contador_frame_der["Derecha"] = 0

    return contador_frame_der

def brazo_izquierda_controls(Segundo_Direccionamiento, Primer_direccionamiento):
    """
    Controla el movimiento del personaje basado en los movimientos del brazo izquierdo.

    Parámetros:
    - Segundo_Direccionamiento: Cadena que indica la dirección secundaria ("Izquierda", "Derecha", "Recto").
    - Primer_direccionamiento: Cadena que indica la dirección primaria ("Arriba", "Abajo", "Izquierda", "Derecha").
    """
    global last_direction_m

    directions_to_keys = {
        ("Arriba", "Izquierda"): [0x57, 0x41],  # W + A
        ("Arriba", "Derecha"): [0x57, 0x44],   # W + D
        ("Arriba", "Recto"): [0x57],           # W
        ("Abajo", "Izquierda"): [0x53, 0x41],  # S + A
        ("Abajo", "Derecha"): [0x53, 0x44],    # S + D
        ("Abajo", "Recto"): [0x53],            # S
        ("Izquierda", "Arriba"): [0x41, 0x57], # A + W
        ("Izquierda", "Abajo"): [0x41, 0x53],  # A + S
        ("Izquierda", "Recto"): [0x41],        # A
        ("Derecha", "Arriba"): [0x44, 0x57],   # D + W
        ("Derecha", "Abajo"): [0x44, 0x53],    # D + S
        ("Derecha", "Recto"): [0x44],          # D
    }

    direction = (Primer_direccionamiento, Segundo_Direccionamiento)
    keys = directions_to_keys.get(direction, [])

    if direction != last_direction_m:
        press_and_hold_movement(keys)
        last_direction_m = direction

def brazo_derecha_controls(Primer_direccionamiento):
    """
    Controla los disparos del personaje basándose en los movimientos del brazo derecho.

    Parámetros:
    - Primer_direccionamiento: Cadena que indica la dirección primaria ("Arriba", "Abajo", "Izquierda", "Derecha").
    """
    global last_direction_t

    directions_to_keys = {
        "Arriba": [0x26],    # Flecha arriba
        "Abajo": [0x28],     # Flecha abajo
        "Izquierda": [0x25], # Flecha izquierda
        "Derecha": [0x27],   # Flecha derecha
    }

    direction = (Primer_direccionamiento)
    keys = directions_to_keys.get(direction, [])

    if direction != last_direction_t:
        press_and_hold_tears(keys)
        last_direction_t = direction

### JSB

In [10]:
""" 
active_keys_m = set()
last_direction_m = None

def simulate_keys(value):
    if value == "ctrl":
        press_key(0x11, 5)
    elif  value == "esc":
        press_key(0x1B, 0.05)    
    elif value == "q":
        press_key(0x51, 0.05)
    elif value == "e":
        press_key(0x45, 0.05)
    elif value == "space":
        press_key(0x20, 0.05)

def press_and_hold_movement(keys_to_press, release_keys=None):
    global active_keys_m

    # Soltar las teclas que ya no están activas
    if release_keys is None:
        release_keys = active_keys_m - set(keys_to_press)
    for key in release_keys:
        ctypes.windll.user32.keybd_event(key, 0, 0x0002, 0)  # Soltar
        active_keys_m.discard(key)

    # Presionar las nuevas teclas
    for key in keys_to_press:
        if key not in active_keys_m:
            ctypes.windll.user32.keybd_event(key, 0, 0, 0)  # Presionar
            active_keys_m.add(key)
"--------------------------------------------------------------------------------------------------------------" 

def pause(resultado_check_nariz, NewValue, Parado_value):
    if (resultado_check_nariz == "Por debajo") and (NewValue[0] == 0):
        simulate_thread = Thread(target=simulate_keys, args=('esc',))
        simulate_thread.start()
        
        NewValue[0] = 1

        if Parado_value[0] == 0:
            Parado_value[0] = 1

        elif Parado_value[0] == 1:
            Parado_value[0] = 0 

    elif (resultado_check_nariz == "Por arriba") and (NewValue[0] == 1):
        NewValue[0] = 0

    return NewValue, Parado_value


def brazo_izquierda_controls(Segundo_Direccionamiento, Primer_direccionamiento):
    global last_direction_m

    # Mapeo de direcciones a teclas
    directions_to_keys = {
        ("Arriba", "Izquierda"): [0x57, 0x41],  # W + A
        ("Arriba", "Derecha"): [0x57, 0x44],   # W + D
        ("Arriba", "Recto"): [0x57],           # W
        ("Abajo", "Izquierda"): [0x53, 0x41],  # S + A
        ("Abajo", "Derecha"): [0x53, 0x44],    # S + D
        ("Abajo", "Recto"): [0x53],            # S
        ("Izquierda", "Arriba"): [0x41, 0x57], # A + W
        ("Izquierda", "Abajo"): [0x41, 0x53],  # A + S
        ("Izquierda", "Recto"): [0x41],        # A
        ("Derecha", "Arriba"): [0x44, 0x57],   # D + W
        ("Derecha", "Abajo"): [0x44, 0x53],    # D + S
        ("Derecha", "Recto"): [0x44],          # D
    }

    # Obtener las teclas actuales
    direction = (Primer_direccionamiento, Segundo_Direccionamiento)
    keys = directions_to_keys.get(direction, [])

    if direction != last_direction_m:
        press_and_hold_movement(keys)
        last_direction_m = direction

def brazo_derecha_controls(Primer_direccionamiento):
    global last_direction_t

    # Mapeo de direcciones a teclas (Flechas de dirección)
    directions_to_keys = {
        "Arriba": [0x26],    # Flecha arriba
        "Abajo": [0x28],     # Flecha abajo
        "Izquierda": [0x25], # Flecha izquierda
        "Derecha": [0x27],   # Flecha derecha
    }

    # Obtener las teclas actuales
    direction = (Primer_direccionamiento)
    keys = directions_to_keys.get(direction, [])

    if direction == "Derecha":
        press_key(0x20, 0.05)
"""

' \nactive_keys_m = set()\nlast_direction_m = None\n\ndef simulate_keys(value):\n    if value == "ctrl":\n        press_key(0x11, 5)\n    elif  value == "esc":\n        press_key(0x1B, 0.05)    \n    elif value == "q":\n        press_key(0x51, 0.05)\n    elif value == "e":\n        press_key(0x45, 0.05)\n    elif value == "space":\n        press_key(0x20, 0.05)\n\ndef press_and_hold_movement(keys_to_press, release_keys=None):\n    global active_keys_m\n\n    # Soltar las teclas que ya no están activas\n    if release_keys is None:\n        release_keys = active_keys_m - set(keys_to_press)\n    for key in release_keys:\n        ctypes.windll.user32.keybd_event(key, 0, 0x0002, 0)  # Soltar\n        active_keys_m.discard(key)\n\n    # Presionar las nuevas teclas\n    for key in keys_to_press:\n        if key not in active_keys_m:\n            ctypes.windll.user32.keybd_event(key, 0, 0, 0)  # Presionar\n            active_keys_m.add(key)\n"--------------------------------------------------

In [11]:
def Camara(Modo):
    """
    Esta función es el núcleo del sistema de control basado en YOLOv8 y captura de cámara. Realiza las siguientes tareas:

    1. Inicializa la configuración para la captura de video desde la cámara y para el modelo YOLOv8 especializado en detección de poses.
    2. Define puntos clave de interés, como las extremidades y articulaciones relevantes, para el análisis.
    3. Procesa cada cuadro capturado por la cámara para:
       - Detectar puntos clave en el cuerpo.
       - Realizar cálculos geométricos entre los puntos clave, como ángulos y distancias.
       - Clasificar las direcciones de los brazos y piernas basándose en estos cálculos.
    4. Controla el movimiento y las acciones del juego simulando pulsaciones de teclas según las posiciones del cuerpo detectadas.
    5. Muestra unas interfaces gráficas para debuggear si es necesario, estás incluye:
       - Una representación gráfica de la pose esquelética detectada.
       - Una consola de texto con mensajes y resultados detallados sobre los cálculos realizados.
    6. Implementa controles para pausar o reanudar las simulaciones de teclas según las posiciones detectadas.
    7. Termina la ejecución si se presiona la tecla "q".

    Parámetros:
    - No recibe parámetros directos; utiliza configuraciones predeterminadas para la cámara y el modelo YOLOv8.

    Componentes clave:
    - `NOMBRES_PUNTOS`: Lista de puntos clave detectados por el modelo YOLOv8.
    - `NOMBRES_PUNTOS_INTERES`: Subconjunto de puntos clave relevantes para las clasificaciones y cálculos.
    - `crear_consola()`: Función para inicializar una consola gráfica donde se muestra información textual.
    - `pause()`: Función que controla la pausa basada en la posición de la nariz.
    - `brazo_izquierda_controls()` y `brazo_derecha_controls()`: Controlan las acciones de los brazos.
    - `pierna_izquierda_controls()` y `pierna_derecha_controls()`: Controlan las acciones de las piernas.
    """
    NOMBRES_PUNTOS = [
        "Nariz", "Ojo izquierdo", "Ojo derecho", "Oreja izquierda", "Oreja derecha",
        "Hombro izquierdo", "Hombro derecho", "Codo izquierdo", "Codo derecho",
        "Muñeca izquierda", "Muñeca derecha", "Cadera izquierda", "Cadera derecha",
        "Rodilla izquierda", "Rodilla derecha", "Tobillo izquierdo", "Tobillo derecho"
    ]

    NOMBRES_PUNTOS_INTERES = [
        "Hombro izquierdo", "Codo izquierdo", "Muñeca izquierda", "Hombro derecho", "Codo derecho", "Muñeca derecha", 
        "Cadera izquierda", "Rodilla izquierda", "Tobillo izquierdo", "Cadera derecha", "Rodilla derecha", "Tobillo derecho"
    ]

    model = YOLO('yolov8n-pose.pt')
    cap = cv2.VideoCapture(1)

    if Modo == "Debug":
        console_image = crear_consola(700, 700) 
    
    # Contador de frames
    frame_counter = 0

    # Pause
    Value = [0]
    Parado = [0]

    # Control de Piernas
    contador_frame_izq = {"Izquierda": 0, "Derecha": 0}
    contador_frame_der = {"Izquierda": 0, "Derecha": 0}
    fps = cap.get(cv2.CAP_PROP_FPS)
    fps_2_seconds = int(fps * 1)

    while True:
        ret, frame = cap.read()

        flipped_frame = cv2.flip(frame, 1)

        results = model(frame, save=False, conf=0.5)

        if Modo == "Debug":
            skeleton_frame = results[0].plot()
            texto_consola = ["Consola de mensajes:"]

        if hasattr(results[0], "keypoints") and results[0].keypoints is not None:
            keypoints_array = results[0].keypoints.data.cpu().numpy()[0]

            if Modo == "Debug": 

                for idx, kp in enumerate(keypoints_array):
                    x, y, conf = int(kp[0]), int(kp[1]), kp[2]

                    if conf <= 0.5:
                        conf = 0

                    if NOMBRES_PUNTOS[idx] in NOMBRES_PUNTOS_INTERES:
                        texto_consola.append(f"{NOMBRES_PUNTOS[idx]}: x={x}, y={y}, confianza={conf:.2f}")

        # Calculos

        resultado_check_nariz = check_Nariz(keypoints_array, NOMBRES_PUNTOS)

        resultado_mano_hombro_izquierda = calcular_distancia_brazo_izquierdo(keypoints_array, NOMBRES_PUNTOS)
        resultado_mano_hombro_derecha = calcular_distancia_brazo_derecho(keypoints_array, NOMBRES_PUNTOS)

        resultado_cadera_pies_izquierda = calcular_distancia_pierna_izquierda(keypoints_array, NOMBRES_PUNTOS)
        resultado_cadera_pies_derecha = calcular_distancia_pierna_derecha(keypoints_array, NOMBRES_PUNTOS)   
        
        # Comentarios de Consola

        if Modo == "Debug":

            texto_consola.append(f"Resultado direccion brazo izquierda: {resultado_mano_hombro_izquierda[0], resultado_mano_hombro_izquierda[1], resultado_mano_hombro_izquierda[2]}")
            texto_consola.append("----------------------------------")
            texto_consola.append(f"Resultado direccion brazo derecha: {resultado_mano_hombro_derecha[0], resultado_mano_hombro_derecha[1], resultado_mano_hombro_derecha[2]}")
            texto_consola.append("----------------------------------")

            texto_consola.append(f"Resultado direccion pierna izquierda: {resultado_cadera_pies_izquierda[0], resultado_cadera_pies_izquierda[1]}")
            texto_consola.append("----------------------------------")
            texto_consola.append(f"Resultado direccion pierna derecha: {resultado_cadera_pies_derecha[0], resultado_cadera_pies_derecha[1]}")
            texto_consola.append("----------------------------------")

            texto_consola.append(f"FPS: {fps_2_seconds}")
            texto_consola.append("----------------------------------")
            texto_consola.append(f"Izquierda: Izquierda: {contador_frame_izq['Izquierda']}, Derecha: {contador_frame_izq['Derecha']}")
            texto_consola.append("----------------------------------")
            texto_consola.append(f"Derecha: Izquierda: {contador_frame_der['Izquierda']}, Derecha: {contador_frame_der['Derecha']}")
            texto_consola.append("----------------------------------")

            texto_consola.append(f"Resultado check Nariz: {resultado_check_nariz}")
            texto_consola.append("----------------------------------")
            texto_consola.append(f"Value: {Value}, Parado: {Parado}")
            texto_consola.append("----------------------------------")

        # Control 

        Value, Parado = pause(resultado_check_nariz, Value, Parado)
                
        if Parado[0] != 1:
            brazo_izquierda_controls(resultado_mano_hombro_izquierda[0], resultado_mano_hombro_izquierda[1])
            brazo_derecha_controls(resultado_mano_hombro_derecha[1])
            contador_frame_izq = pierna_izquierda_controls(resultado_cadera_pies_izquierda[0], contador_frame_izq, fps_2_seconds)
            contador_frame_der = pierna_derecha_controls(resultado_cadera_pies_derecha[0], contador_frame_der, fps_2_seconds)
                
        if Modo == "Debug":        
            console_image[:] = 255 
            escribir_en_consola(console_image, texto_consola)

            cv2.imshow('Consola de Texto', console_image)
            cv2.imshow('Esqueleto Invertido', skeleton_frame)
        
        else: 
            cv2.imshow('YOLOv8 - Camara', flipped_frame)

        frame_counter += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [12]:
if __name__ == "__main__":
    """
    Punto de entrada principal del programa. Ejecuta el menú gráfico en un hilo separado.

    Descripción:
    - Inicia un hilo independiente para la función `run_menu`, configurado como demonio,
      lo que significa que se cerrará automáticamente cuando el programa principal termine.
    - Esto permite que el menú gráfico funcione de manera concurrente con otras tareas.

    Detalles:
    - `menu_thread`: Hilo que ejecuta la función `run_menu` (debe estar definida previamente).
    - `daemon=True`: Configura el hilo como demonio para que no bloquee la terminación del programa principal.
    - `menu_thread.start()`: Inicia la ejecución del hilo.
    """
    menu_thread = threading.Thread(target=run_menu, daemon=True)
    menu_thread.start()


0: 480x640 (no detections), 140.0ms
Speed: 5.0ms preprocess, 140.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 148.0ms
Speed: 7.0ms preprocess, 148.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 146.0ms
Speed: 3.0ms preprocess, 146.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 219.0ms
Speed: 3.0ms preprocess, 219.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 141.0ms
Speed: 2.0ms preprocess, 141.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Ventana 'YOLOv8 - Camara' encontrada

Ventana 'YOLOv8 - Camara' encontrada
0: 480x640 1 person, 158.0ms
Speed: 8.0ms preprocess, 158.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 136.0ms
Speed: 2.0ms preprocess, 136.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Ventana 'Binding of Isa

## Parte de Mediapipe

In [13]:
"""
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import math

# Función para clasificar el ángulo
def clasificar_angulo_dinamico(x1, y1, x2, y2):
    # Calcular el ángulo en radianes
    angulo_radianes = math.atan2((y2 - y1), x2 - x1)

    # Convertir a grados
    angulo_grados = math.degrees(angulo_radianes)

    # Asegurarse de que el ángulo esté entre 0 y 360
    if angulo_grados < 0:
        angulo_grados += 360

    # Clasificar el ángulo
    if 70 <= angulo_grados <= 110:
        return f"Recto (Ángulo: {angulo_grados:.1f})"
    elif angulo_grados > 110:
        return f"Derecha (Ángulo: {angulo_grados:.1f})"
    elif angulo_grados < 70:
        return f"Izquierda (Ángulo: {angulo_grados:.1f})"

# Configurar Mediapipe FaceLandmarker
base_options = python.BaseOptions(model_asset_path=r'C:/Users/Mario/Desktop/face_landmarker.task')
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=1
)
detector = vision.FaceLandmarker.create_from_options(options)

def procesar_frame(frame_invertido):
    # Obtener dimensiones del frame
    frameX, frameY, _ = frame_invertido.shape

    # Adapta el formato del frame para Mediapipe
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_invertido)

    # Detección de puntos faciales
    detection_result = detector.detect(image)
    face_landmarks_list = detection_result.face_landmarks

    if face_landmarks_list:
        face_landmarks = face_landmarks_list[0]  # Solo trabajamos con el primer rostro

        # Coordenadas en píxeles de los puntos 10 y 152
        punto_10_x = face_landmarks[10].x * frameY
        punto_10_y = face_landmarks[10].y * frameX
        punto_152_x = face_landmarks[152].x * frameY
        punto_152_y = face_landmarks[152].y * frameX

        # Calcular y clasificar el ángulo
        resultado = clasificar_angulo_dinamico(punto_10_x, punto_10_y, punto_152_x, punto_152_y)
        return resultado

    # Si no se detectan puntos, devolver None
    return "No se detectaron puntos faciales"

# Captura de cámara
vid = cv2.VideoCapture(0)

while True:      
    ret, frame = vid.read()
    if not ret:
        print("Error al capturar el frame.")
        break

    # Invertir el frame
    flipped_frame = cv2.flip(frame, 1)

    # Llamar a la función procesar_frame
    resultado = procesar_frame(flipped_frame)

    # Imprimir el resultado
    print(resultado)

    cv2.imshow('Vid', flipped_frame)

    # Salir con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar recursos
vid.release()
cv2.destroyAllWindows()
"""

'\nimport cv2\nimport mediapipe as mp\nfrom mediapipe.tasks import python\nfrom mediapipe.tasks.python import vision\nimport math\n\n# Función para clasificar el ángulo\ndef clasificar_angulo_dinamico(x1, y1, x2, y2):\n    # Calcular el ángulo en radianes\n    angulo_radianes = math.atan2((y2 - y1), x2 - x1)\n\n    # Convertir a grados\n    angulo_grados = math.degrees(angulo_radianes)\n\n    # Asegurarse de que el ángulo esté entre 0 y 360\n    if angulo_grados < 0:\n        angulo_grados += 360\n\n    # Clasificar el ángulo\n    if 70 <= angulo_grados <= 110:\n        return f"Recto (Ángulo: {angulo_grados:.1f})"\n    elif angulo_grados > 110:\n        return f"Derecha (Ángulo: {angulo_grados:.1f})"\n    elif angulo_grados < 70:\n        return f"Izquierda (Ángulo: {angulo_grados:.1f})"\n\n# Configurar Mediapipe FaceLandmarker\nbase_options = python.BaseOptions(model_asset_path=r\'C:/Users/Mario/Desktop/face_landmarker.task\')\noptions = vision.FaceLandmarkerOptions(\n    base_option

: 